In [4]:
import os
from selenium import webdriver
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re


In [5]:
os.environ['PATH'] += r";C:/DriversSelenium"
edge_driver_path = "C:/DriversSelenium/MicrosoftWebDriver.exe"
driver = webdriver.Edge(executable_path=edge_driver_path)
# url = driver.get("https://www.linkedin.com/jobs/collections/recommended/?currentJobId=3787000603&eBP=CwEAAAGNQiS7jzP4PM6jfClBdbBcA5Kj7OKMjBCRUjHA-HdI-0Ex1HjemMeguJUizAVl3d-Ykh2nrMpULtQDydr99cbF1pjbIKyxy4xwiDJyfZTlRr6ak0WMVLQCK7F4itZbKzqrUKkT-lZx-BqobFR6azzQO-vvsl367EqX0SuOWMf5PGA3wasRXak92OXGs95U8IZQFF_CWFjFqXgiSiSgkYzHXNisbCzFQEbwPoclZSkrpcwfsKyPaFLVxuuHdSeydE4v6qzL75AUbYcgPZgc0JUBgbsFMpwuYcNHCqGi6yIGu6NxeH7MgWz11Vand6Q96xtz6_aImig97aKslBiXRv1V-ziSzBZyVq97hbSkLCivMBkw6UpPMSlbVwBhFU8&refId=6t7GHxpiqOK%2BVQXbUU%2BdAQ%3D%3D&trackingId=%2FRXN7kZXNSs4Yhl4W0YsRw%3D%3D")
driver.get('https://www.linkedin.com/jobs/search/?currentJobId=3797029583&geoId=102787409&keywords=data%20science&location=Morocco&origin=JOB_SEARCH_PAGE_KEYWORD_AUTOCOMPLETE&refresh=true')

In [ ]:
jobs = []
companies = []
cities = []
niveau_experience = []
salaries = []
work_type = [] #full-time, part-time
work_model = [] #hybride, on-site will take it from city hihihi
skills = []

for i in range(1,18):#Iteration through pages
   driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
   try:
    #button = driver.find_element(By.XPATH, f'//button[@aria-label="Page 2"]')#find button
         button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f'//button[@aria-label="Page {i}"]')) )
         button.click()
         time.sleep(2)
   except:
         print("Error clicking page")

   all_jobs = driver.find_elements(By.CLASS_NAME, 'full-width.artdeco-entity-lockup__title.ember-view')
   for job in all_jobs:
         try:
            #Jobs
            job_title = job.find_element(By.TAG_NAME, 'strong').text
            jobs.append(job_title)
            driver.implicitly_wait(10)
            parent_div = job.find_element(By.XPATH, '..')

            #Companies
            company=parent_div.find_element(By.CLASS_NAME,'job-card-container__primary-description').text
            companies.append(company)
            driver.implicitly_wait(10)

            #Cities
            city = parent_div.find_element(By.CLASS_NAME,'job-card-container__metadata-item').text
            cities.append(city)
            driver.implicitly_wait(10)

            #Work model from City
            match = re.search(r'\((.*?)\)',city)
            if match:
                  content= match.group(1)
                  work_model.append(content)
            else:
                  work_model.append(np.nan)
                  driver.implicitly_wait(10)

            parent_div.click()
            try:
                 #Skills
                 skill = driver.find_element(By.CSS_SELECTOR, 'a.app-aware-link.job-details-how-you-match__skills-item-subtitle.t-14.overflow-hidden').text
                 skills.append(skill) 
                 driver.implicitly_wait(10) 
                 
                 #Salaries
                 salary = driver.find_element(By.CLASS_NAME, "jobs-details__salary-main-rail-card").text
                 if salary == '':
                      salaries.append(np.nan)
                      driver.implicitly_wait(10)
                  
                 #Work_type and niveau_experience
                 span = driver.find_elements(By.CLASS_NAME, "job-details-jobs-unified-top-card__job-insight-view-model-secondary")
                 for i, s in enumerate(span, start=0):
                        text = s.text
                        if i == 0:
                              work_type.append(text)
                              driver.implicitly_wait(10)
                        if i == 1:
                              niveau_experience.append(text)
                              driver.implicitly_wait(10)
                        if not work_type:
                              work_type.append(np.nan)

                        if not niveau_experience:
                              niveau_experience.append(np.nan)
            
            except:
                 print("Not found")

         except:
            print('Not done')
   i+=1#increment pages
    


In [50]:
print(jobs)
print(companies)
print(cities)
print(work_type)
print(work_model)
print(salaries)
print(niveau_experience)
print(skills)
len(jobs)


['Manager Data et Analytics F/H', 'Data Analyst Senior', 'Data Engineer F/H', 'Senior Data Scientist', 'Backend Senior Developer H/F - Casablanca', 'Senior Data Scientist (H/F)', 'Data Analyst', 'Administrateur Systèmes et Bases de Données', 'Développeur ERP H/F', 'Consultant Odoo', 'Business Developer', 'Tech Lead Full Stack', 'Glovo Africa Graduate Program - Morocco', 'Data Engineer/ TALEND', 'Stagiaire VBA H/F', 'Data Engineer', 'Consultant Talend ETL (F/H)', 'Développeur FullStack .NET Angular | Devoteam Maroc Nearshore', 'Digital Strategy Consultant (Technology Digital Advantage)', '(Junior/Senior) Software Engineer - X Delivery', 'FULLSTACK DEVELOPER', 'Network Engineer Intern', 'Prompt Engineer ', 'Senior Software Engineer', "Stage en informatique & Systèmes d'Information", 'Chef de projet Web F/H', 'Fresh Specialist - Morocco', 'Consulting Data Manager', 'Manager', 'Responsable Qualification Produit H/F ', 'Data Architect (H/F)', 'Consultant Expérimenté en Business Intelligence

257

In [51]:
jobs_col = pd.DataFrame(jobs,columns=['Job'])
companies_col = pd.DataFrame(companies,columns=['Company'])
cities_col = pd.DataFrame(cities,columns=['City'])
work_type_col= pd.DataFrame(work_type,columns=['Work Type'])
work_model_col = pd.DataFrame(work_model,columns=['Work Model'])
salaries_col = pd.DataFrame(salaries,columns=['Salary'])
niveau_experience_col = pd.DataFrame(niveau_experience,columns=['Experience Level'])
skills_col = pd.DataFrame(skills,columns=['Skills'])

In [52]:
final_result = pd.concat([jobs_col, companies_col, cities_col, work_type_col, work_model_col, salaries_col, niveau_experience_col, skills_col], axis=1)
final_result

,Job,Company,City,Work Type,Work Model,Salary,Experience Level,Skills
0,Manager Data et Analytics F/H,Deloitte,"Mechouar, Casablanca-Settat, Morocco (Hybrid)",Full-time,Hybrid,NaN,NaN,"Coaching, DWH, ETL, Esprit, Microsoft Business..."
1,Data Analyst Senior,Leyton,Casablanca Metropolitan Area (On-site),Full-time,On-site,NaN,Mid-Senior level,"CRM, Microsoft Excel, PowerPoint, Python, Sale..."
2,Data Engineer F/H,Zenika,Casablanca Metropolitan Area (Hybrid),Full-time,Hybrid,NaN,Mid-Senior level,"Apache Kafka, Apache Spark, Google Cloud Platf..."
3,Senior Data Scientist,Leyton,"Casablanca-Settat, Morocco (Hybrid)",Full-time,Hybrid,NaN,Associate,"Decision Trees, Linear Regression, Python, Sup..."
4,Backend Senior Developer H/F - Casablanca,Eskimoz,Casablanca Metropolitan Area (Hybrid),Full-time,Hybrid,NaN,Mid-Senior level,"PHP, الإنجليزية, الاتصالات ثنائية اللغة, التوا..."
...,...,...,...,...,...,...,...,...
252,Ingénieur Support (H/F),Sofrecom Maroc,"Rabat, Rabat-Salé-Kenitra, Morocco (Hybrid)",NaN,Hybrid,NaN,NaN,NaN
253,Chef de Projet Technique (.NET),SQLI,"Agdal, Fès-Meknès, Morocco (On-site)",NaN,On-site,NaN,NaN,NaN
254,Développeur Java / JEE -Senior- CDI - F/H,Eviden,"Prefecture of Casablanca, Casablanca-Settat, M...",NaN,Hybrid,NaN,NaN,NaN
255,"Conseiller(e) Commercial(e) Private ""Stage""",Avito.ma - Frontier Digital Ventures,"Casablanca-Settat, Morocco (On-site)",NaN,On-site,NaN,NaN,NaN


In [53]:
final_result.to_csv('D:/webScrapping/datasetLinkdin.csv', index=False)